In [71]:
bits = 128

def safe_prime(nbits):
    while True:
        p = random_prime(2^nbits-1, false, 2^(nbits-1))
        if ZZ((p+1)/2).is_prime():
            return p
        
N = safe_prime(bits)
F = FiniteField(N)
k = F.random_element()
g = F.multiplicative_generator()
I = 'Bob'

import hashlib
from sage.crypto.util import ascii_to_bin

def H(*args):
    hs = hashlib.sha1()
    for a in args:
        hs.update(str(a))
    return hs.hexdigest()

def hash_to_int(hs):
    return ZZ('0x' + hs)

In [72]:
p = getrandbits(bits)
s = getrandbits(32)
x = hash_to_int(H(s, p))
v = g^x
(s, v)

(1598876508L, 53494898264320744629372320720535041848)

In [73]:
a = getrandbits(bits)
A = g^a
(I, A)

('Bob', 194915951129395064026584438670225872096)

In [74]:
b = getrandbits(bits)
B = k * v + g^b
(s, B)

(1598876508L, 42498612804040704006477076953938202030)

In [75]:
u = hash_to_int(H(A, B))

In [76]:
x = hash_to_int(H(s, p))
S_user = (B - k*g^x)^(a + u*x)
K_user = H(S)
K_user

'7db40c734db299fd9c564f63d18d0f9130622886'

In [77]:
S_host = (A * v^u)^b
K_host = H(S)
K_host

'7db40c734db299fd9c564f63d18d0f9130622886'

In [78]:
K_user == K_host

True